In [1]:
import requests
import datetime
import time
import csv
import json
import pandas as pd

In [13]:
page_id = "hkgourmetkitchen"

app_id = "142436546472537"
app_secret = "1d0cad94687a3b10366b65c79ab1abc1"

access_token = app_id + "|" + app_secret


In [3]:
def request_until_succeed(url):
    success = False
    while success is False:
        try: 
            req = requests.get(url)
            if req.status_code == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)
            print("Error for URL %s: %s" % (url, datetime.datetime.now()))
            print("Retrying.")

    return req


In [4]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):

    # Construct the URL string; see http://stackoverflow.com/a/37239851 for
    # Reactions parameters
    base = "https://graph.facebook.com/v2.11"
    node = "/%s/posts" % page_id 
    fields = "/?fields=message,link,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions" + \
            ".limit(0).summary(true)"
    parameters = "&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + fields + parameters

    # 取得data
    data = request_until_succeed(url).json()
    return data

In [5]:
def getReactionsForStatus(status_id, access_token):

    # See http://stackoverflow.com/a/37239851 for Reactions parameters
        # Reactions are only accessable at a single-post endpoint

    base = "https://graph.facebook.com/v2.11"
    node = "/%s" % status_id
    reactions = "/?fields=" \
            "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
            ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
            ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
            ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
            ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
            ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
    parameters = "&access_token=%s" % access_token
    url = base + node + reactions + parameters

    # 取得data
    data = request_until_succeed(url).json()
    return data


In [6]:
def processFacebookPageFeedStatus(status, access_token):

    # 要去確認抓到的資料是否為空
    status_id = status['id']
    status_type = status['type']
    if 'message' not in status.keys():
        status_message = ''
    else:
        status_message = status['message']
    if 'name' not in status.keys():
        link_name = ''
    else:
        link_name = status['name']
    link = status_id.split('_')
    
    # 此連結可以回到該臉書上的post
    status_link = 'https://www.facebook.com/'+link[0]+'/posts/'+link[1]

    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    # 根據所在時區 TW +8
    status_published = status_published + datetime.timedelta(hours=8)
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') 
    
    # 要去確認抓到的資料是否為空
    if 'reactions' not in status:
        num_reactions = 0
    else:
        num_reactions = status['reactions']['summary']['total_count']
    if 'comments' not in status:
        num_comments = 0
    else:
        num_comments = status['comments']['summary']['total_count']
    if 'shares' not in status:
        num_shares = 0
    else:
        num_shares = status['shares']['count']

    def get_num_total_reactions(reaction_type, reactions):
        if reaction_type not in reactions:
            return 0
        else:
            return reactions[reaction_type]['summary']['total_count']
    
    # 取得該篇文章的 reactions like,love,wow,haha,sad,angry數目
    reactions = getReactionsForStatus(status_id, access_token)
    
    num_loves = get_num_total_reactions('love', reactions)
    num_wows = get_num_total_reactions('wow', reactions)
    num_hahas = get_num_total_reactions('haha', reactions)
    num_sads = get_num_total_reactions('sad', reactions)
    num_angrys = get_num_total_reactions('angry', reactions)
    num_likes = get_num_total_reactions('like', reactions)

    # 回傳tuple形式的資料
    return (status_id, status_message, link_name, status_type, status_link,
            status_published, num_reactions, num_comments, num_shares,
            num_likes, num_loves, num_wows, num_hahas, num_sads, num_angrys)

In [7]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    # all_statuses 用來儲存的list,先放入欄位名稱
    all_statuses = [('status_id', 'status_message', 'link_name', 'status_type', 'status_link',
            'status_published', 'num_reactions', 'num_comments', 'num_shares',
            'num_likes', 'num_loves', 'num_wows', 'num_hahas', 'num_sads', 'num_angrys')]
    
    has_next_page = True 
    num_processed = 0   # 計算處理多少post
    scrape_starttime = datetime.datetime.now()

    print("Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime))

    statuses = getFacebookPageFeedData(page_id, access_token, 100)

    while has_next_page:
        for status in statuses['data']:

            # 確定有 reaction 再把結構化後的資料存入 all_statuses
            if 'reactions' in status:
               all_statuses.append(processFacebookPageFeedStatus(status,access_token))

            # 觀察爬取進度,每處理100篇post,就輸出時間,
            num_processed += 1
            if num_processed % 100 == 0:
                print("%s Statuses Processed: %s" % (num_processed, datetime.datetime.now()))

        # 每超過100個post就會有next,可以從next中取得下100篇, 直到沒有next
        if 'paging' in statuses.keys() and 'next' in statuses['paging'].keys():
            statuses = request_until_succeed(statuses['paging']['next']).json()
        else:
            has_next_page = False

    print("\nDone!\n%s Statuses Processed in %s" % \
        (num_processed, datetime.datetime.now() - scrape_starttime))
    
    return all_statuses

In [14]:
all_statuses = scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping hkgourmetkitchen Facebook Page: 2018-01-23 19:27:51.648051



KeyboardInterrupt: 

In [10]:
df = pd.DataFrame(all_statuses[1:], columns=all_statuses[0])

In [11]:
df.tail()

,status_id,status_message,link_name,status_type,status_link,status_published,num_reactions,num_comments,num_shares,num_likes,num_loves,num_wows,num_hahas,num_sads,num_angrys
393,216259018448153_369350723138981,,Dr Reborn $987 療程 (優惠已於2014年1月31日完結),photo,https://www.facebook.com/216259018448153/posts...,2012-09-26 19:01:24,2,0,0,2,0,0,0,0,0
394,216259018448153_368599479880772,全港首創劑量保證療程! \nBotox V面、Restylane高鼻、豐下巴或蘋果肌，\n推...,Dr Reborn $987 療程 (優惠已於2014年1月31日完結),photo,https://www.facebook.com/216259018448153/posts...,2012-09-24 17:22:51,7,0,3,7,0,0,0,0,0
395,216259018448153_321517527922301,阿嬌\r\n連續4年成為Dr Reborn代言人,[Dr Reborn V面‧纖腿代言人 - 阿嬌(鍾欣桐)],photo,https://www.facebook.com/216259018448153/posts...,2012-05-29 11:19:57,4,0,3,4,0,0,0,0,0
396,216259018448153_259960077411380,Dr Reborn 採用高水準、國際性醫學美容產品，如美國Botox、英國Dysport、瑞...,[DrReborn 專業榮譽] 國際藥廠認証 銷量NO.1,photo,https://www.facebook.com/216259018448153/posts...,2012-02-22 17:57:11,12,0,0,12,0,0,0,0,0
397,216259018448153_376949192322423,堅持以「專業‧安全‧有效」為宗旨 得到大眾嘅信賴與支持\n \nDr Reborn 榮獲國...,DR REBORN,photo,https://www.facebook.com/216259018448153/posts...,2012-01-28 13:35:26,0,0,0,0,0,0,0,0,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 15 columns):
status_id           398 non-null object
status_message      398 non-null object
link_name           398 non-null object
status_type         398 non-null object
status_link         398 non-null object
status_published    398 non-null object
num_reactions       398 non-null int64
num_comments        398 non-null int64
num_shares          398 non-null int64
num_likes           398 non-null int64
num_loves           398 non-null int64
num_wows            398 non-null int64
num_hahas           398 non-null int64
num_sads            398 non-null int64
num_angrys          398 non-null int64
dtypes: int64(9), object(6)
memory usage: 46.7+ KB


In [13]:
path = 'post/'+page_id+'_post.csv'
df.to_csv(path,index=False,encoding='utf8')